# Experiment 1
I try to process a specific text here.

In [12]:
text = '__团队成员#1__先生是公司创始人,也是中国最有影响力的商界领袖之一。1982年,__团队成员#1__先生于华南理工大学毕业,进入TCL的前身-TTK家庭电器有限公司。1985年,他担任新成立的TCL通讯设备公司总经理,创立了TCL品牌。2003年,__团队成员#1__担任TCL集团股份有限公司董事长兼CEO,随后TCL集团整体上市。在他的领导下,2004年TCL一举收购了法国汤姆逊全球彩电业务与阿尔卡特全球手机业务。目前TCL集团已经成为拥有6万名员工,业务遍及全球80多个国家和地区。2013年,TCL集团营业总收入超过855亿元,液晶电视全球销量1766万台,实际产量全球第三,品牌销售全球第三;TCL手机全球销量5520万台,行业排名全球第五。2012年__团队成员#1__被新华网评为“最具社会责任感企业家”;2011年荣获《中国企业家》“最具影响力的25位企业领袖”终身成就奖;2009年被评为“CCTV中国经济年度人物十年商业领袖”;2008年获改革开放30年经济人物称号;2004年被评为Fortune杂志“亚洲年度经济人物”、TIME杂志和CNN全球最具影响力的25名商界人士,同年法国总统希拉克向__团队成员#1__先生颁发了法国国家荣誉勋章。__团队成员#1__是中共第十六大代表,第十届、第十一届、第十二届全国人大代表。__团队成员#1__担任的社会职务包括:中国电子视像行业协会会长;中国国际商会副会长;全国工商联执行委员、广东省工商联(总商会)副主席。'

In [157]:
print(text)

__团队成员#2__先生,现任TCL集团股份有限公司执行董事、总裁(COO)。1963年4月出生,博士,毕业于西安交通大学。薄先生1988年至1993年间任陕西财经学院贸易经济系副主任,1993年5月至2000年5月间任深圳航空公司总会计师,是深圳航空公司创始人之一。薄先生2000年5月至2004年4月间任TCL信息产业集团副总裁、财务总监,2004年4月至2005年1月间任TCL集团部品事业本部副总裁,2005年1月至2005年10月间任TCL集团股份有限公司人力资源部部长、总裁办主任,2005年10月至2006年11月间任TCL集团股份有限公司控股子公司TTE Corporation执行副总裁,2006年6月至2007年9月间任TCL集团股份有限公司人力资源总监,2006年8月至2007年10月间任TCL集团股份有限公司副总裁,2007年10月至2008年6月间任高级副总裁,2008年6月至2011年6月,任TCL集团股份有限公司首席运营官,2011年6月至今任TCL集团股份有限公司总裁(COO),2012年12月24日起任深圳市华星光电技术有限公司CEO。__团队成员#2__先生现兼任TCL多媒体科技控股有限公司执行董事、深圳市华星光电技术有限公司董事、翰林汇信息产业股份有限公司董事长、TCL教育网有限公司董事长、电大在线远程教育技术有限公司副董事长。__团队成员#2__先生于二零一四年十月二十三日调任为TCL多媒体科技控股有限公司之执行董事，并获委任为策略执行委员会主席。__团队成员#2__先生获委任为TCL多媒体科技控股有限公司首席执行官,自二零一五年六月二十五日起生效。


In [14]:
import jieba
import re
import chardet
from gensim.models import KeyedVectors

# Preprocess Text
+ remove punctuations
+ remove stopwords
+ to lower letter for regconition later
+ generate N-grams to link entity

In [15]:
def removeStopWords(seglist):
    stopwords = {}
    fstop = open('data/stopwords_cn.txt', 'r', encoding='utf-8', errors='ignore')
    for w in fstop:
        stopwords[w.strip()] = w.strip()

    fstop.close()
    stopwords[' '] = ' '
    
    segListSanitized = []

    for word in seglist:
        if word not in stopwords:
            segListSanitized.append(word)
    return segListSanitized

In [16]:
def preprocess(text):
    # remove punctuations
    text = re.sub(r"[\s+\.\!\/_,$%^*()?;；:【】+\"\']+|[+——！，;:。？、~@#￥%……&*（）]+", " ", text)
    text = text.lower()
    # seperate words
    words = jieba.cut(text, cut_all=False)
    seglist = list(words)
    # remove stopwords
    segListSanitized = removeStopWords(seglist)
    print(f'Before sanitize, len: {len(seglist)}. After sanitize, len: {len(segListSanitized)}')

    return segListSanitized

In [17]:
def getNgrams(wordList, n):
    '''
    This function only generete N-Grams
    '''
    output = set()
    for i in range (len(wordList) - n + 1):
        n_gram_temp = "".join(wordList[i:i+n])
        output.add(n_gram_temp)
    return output

In [31]:
def generateNgrams(wordList, n):
    '''
    This function genereates [1, N]-Grams
    '''
    result = set()
    for i in range(n):
        temp = getNgrams(wordList, i+1)
        result = result | temp
        print(f'i = {i}, len: {len(temp)}')
    
    return result

In [160]:
segListSanitized = preprocess(text)
output = generateNgrams(segListSanitized, 3)
print(len(output))

Before sanitize, len: 330. After sanitize, len: 264
i = 0, len: 108
i = 1, len: 172
i = 2, len: 209
489


# Linking Using Word Embedding

In [36]:
model = KeyedVectors.load('./test_50.bin')

In [44]:
import numpy as np
def calculate_cosine_similarity(a, b):
    vector_a = np.mat(a)
    vector_b = np.mat(b)
    num = float(vector_a * vector_b.T)
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    cos = num / denom
    sim = 0.5 + 0.5 * cos
    return sim

In [45]:
def generateEmbeddings(name):
    words = jieba.cut(name, cut_all=False)
    word_list = list(words)
    v = np.zeros((200))
    for word in word_list:
        if word in model.vocab:
            v += model[word]
    
    v /= len(v)
    return v

In [163]:
def preprocess_entity_list(df, model):
    '''
    df: dafaframe
    model: word embedding model
    '''
    
    df['embeddings'] = ''
    for index, row in df.iterrows():
        # df.loc[index, 'embeddings'] = z
        name = row['name']
        if isinstance(name, float):
            continue
        name = name.lower()
        if name in model.vocab:
            vec = model[name]
        else:
            vec = generateEmbeddings(name)
        df.set_value(index, 'embeddings', vec)

    print(df.head())
    return df

## Link School

In [164]:
import pandas as pd
path0 = 'data/chinese_university_list.csv'

In [165]:
school_df = pd.read_csv(path0, header=None, delimiter=",", skiprows=4, names=["rank", "name", "code", "department", "city", "level", "notes"])
school_df = pd.DataFrame(school_df, columns=['rank', 'name'])
print(school_df.head())

  rank    name
0    1    北京大学
1    2  中国人民大学
2    3    清华大学
3    4  北京交通大学
4    5  北京工业大学


In [170]:
def linkSchool(output, school_df, model, threshold):
    for term in output:
        if term in model.vocab:
            term_vec = model[term]
            school_candidate = dict()
            for index, row in school_df.iterrows():
                name = row['name']
                if isinstance(name, float):
                    continue
                name_vec = row['embeddings']
                sim = calculate_cosine_similarity(term_vec, name_vec)
                if (sim > threshold):
                    school_candidate[name] = sim
            if len(school_candidate) == 0:
                continue
            school_candidate = sorted(school_candidate.items(), key=lambda item:item[1], reverse=True)
            print(f'university entity found: {term}->{school_candidate[0][0]}')

In [171]:
school_df = preprocess_entity_list(school_df, model)

c:\users\梁育诚\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


  rank    name                                         embeddings
0    1    北京大学  [0.037607472, 0.039331738, 0.024360932, -0.023...
1    2  中国人民大学  [0.029720008, -0.04914518, -0.06820664, 0.0211...
2    3    清华大学  [0.015153781, 0.046060637, -0.0013895197, -0.0...
3    4  北京交通大学  [0.019046584, -0.0035929743, -0.022275247, -0....
4    5  北京工业大学  [0.01881563, -0.007572121, -0.009649365, -0.00...


In [57]:
linkSchool(output, school_df, model, 0.9)

university entity found: 华南理工大学->华南理工大学


In [172]:
linkSchool(output, school_df, model, 0.9)

university entity found: 西安交通大学->西安交通大学
university entity found: 财经学院->宁波财经学院


## Link Company

In [84]:
import pandas as pd
import math
path4 = 'data/company_list_ch.csv'

In [145]:
company_ch_df = pd.read_csv(path4, header=None, delimiter=",", skiprows=2, names=['rank', 'name', 'Location', 'Income'])
company_ch_df = pd.DataFrame(company_ch_df, columns=['rank', 'name'])
print(company_ch_df.head())

   rank             name
0     1   中国石油化工集团有限公司\n
1     2  中国石油天然气集团有限公司\n
2     3       国家电网有限公司\n
3     4     中国建筑股份有限公司\n
4     5   中国工商银行股份有限公司\n


In [146]:
def calculate_IDF(df):
    company_num = 0
    m = dict()
    for index, row in df.iterrows():
        name = row['name']
        name = re.sub(r"[\s+\.\!\/_,$%^*()?;；:【】+\"\']+|[+——！，;:。？、~@#￥%……&*（）]+", " ", name)
        name = name.lower()
        if isinstance(name, float) or name == " ":
            continue
        company_num += 1
        words = jieba.cut(name, cut_all=False)
        word_list = list(words)
        for word in word_list:
            if word in m.keys():
                m[word] +=1
            else:
                m[word] = 1
                
    print(f'company number: {company_num}, dict size: {len(m)}')
    # print(m)
    
    idf = dict()
    
    for (k, v) in m.items():
        idf[k] = math.log(((1+company_num) / v), 10)
    
    # Normalize min-max
    v = idf.values()
    max_value = max(v)
    min_value = min(v)
    denom = max_value - min_value
    for (k, v) in idf.items():
        idf[k] = (idf[k] - min_value) / denom

    print(idf)
    return idf

In [147]:
idf = calculate_IDF(company_ch_df)

company number: 500, dict size: 622
{'中国石油化工集团': 1.0, '有限公司': 0.05513610495796486, ' ': 0.0, '中国石油天然气集团': 1.0, '国家电网': 1.0, '中国': 0.3738793760288948, '建筑': 0.8283487347101395, '股份': 0.3551611915126718, '中国工商银行': 1.0, '平安保险': 1.0, '集团': 0.13730643414831753, '中国建设银行': 1.0, '中国农业银行': 1.0, '上海': 0.5876638788101248, '汽车': 0.8283487347101395, '集团股份': 0.46346108202223235, '中国银行': 1.0, '人寿保险': 0.7485354415526261, '公司': 0.36291040184728496, '铁路': 0.8283487347101395, '工程': 1.0, '中国移动通信集团': 1.0, '铁道': 1.0, '华为': 1.0, '投资': 0.4358167708877228, '控股': 0.4123361211898751, '海洋': 1.0, '石油': 0.8917001094904394, '国家开发银行': 1.0, '华润': 1.0, '苏宁': 1.0, '控股集团': 0.4052329109729049, '东风汽车': 1.0, '第一': 1.0, '汽车集团': 0.7485354415526261, '交通': 0.7200488442005789, '建设': 0.5876638788101248, '中化': 1.0, '集团公司': 0.7834002189808787, '太平洋': 1.0, '中国邮政': 1.0, '国家': 0.8283487347101395, '能源': 0.5399499154015097, '有限责任': 0.3856769565408303, '南方电网': 1.0, '五矿': 1.0, '正威': 1.0, '国际': 0.6253439344762688, '人民保险': 1.0, '北京': 0.5768

In [148]:
def generateCompanyEmbeddings(name, idf):
    words = jieba.cut(name, cut_all=False)
    word_list = list(words)
    v = np.zeros((200))
    for word in word_list:
        if word in model.vocab:
            # emb(term) * IDF
            # print(f'word: {word}')
            # print(idf[word])
            v += model[word] * idf[word]
    
    v /= len(v)
    return v

In [152]:
def preprocess_company_entity_list(df, model, idf):
    '''
    df: dafaframe
    model: word embedding model
    '''
    
    df['embeddings'] = ''
    for index, row in df.iterrows():
        name = row['name']
        name = re.sub(r"[\s+\.\!\/_,$%^*()?;；:【】+\"\']+|[+——！，;:。？、~@#￥%……&*（）]+", " ", name)
        name = name.lower()
        if isinstance(name, float):
            continue
        if name in model.vocab:
            vec = model[name]
        else:
            vec = generateCompanyEmbeddings(name, idf)
        df.set_value(index, 'embeddings', vec)

    print(df.head())
    return df

In [153]:
def linkCompany(output, company_df, model, threshold):
    for term in output:
        if len(term) <= 1:
            continue
        if term in model.vocab:
            term_vec = model[term]
            company_candidate = dict()
            for index, row in company_df.iterrows():
                name = row['name']
                if isinstance(name, float):
                    continue
                name_vec = row['embeddings']
                sim = calculate_cosine_similarity(term_vec, name_vec)
                if (sim > threshold):
                    company_candidate[name] = sim
            if len(company_candidate) == 0:
                continue
            company_candidate = sorted(company_candidate.items(), key=lambda item:item[1], reverse=True)
            print(f'company entity found: {term}->{company_candidate[0][0]}, sim = {company_candidate[0][1]}')

In [154]:
company_ch_df = preprocess_company_entity_list(company_ch_df, model, idf)

c:\users\梁育诚\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


   rank             name                                         embeddings
0     1   中国石油化工集团有限公司\n  [0.00020857858937233686, 0.0002797483908943832...
1     2  中国石油天然气集团有限公司\n  [0.0004169972753152251, -7.988460594788194e-05...
2     3       国家电网有限公司\n  [0.00040219956543296575, -0.000430932913441211...
3     4     中国建筑股份有限公司\n  [0.00030216909712180493, -0.000533461889717727...
4     5   中国工商银行股份有限公司\n  [0.00018973188241943717, -0.000447552979458123...


In [155]:
linkCompany(output, company_ch_df, model, 0.9)

company entity found: tcl->TCL集团股份有限公司
, sim = 0.9610930736775853
company entity found: 有限公司->中运富通控股集团有限公司
, sim = 0.9063590600834653
company entity found: 集团股份有限公司->中运富通控股集团有限公司
, sim = 0.9037290995606039
company entity found: 企业->万科企业股份有限公司
, sim = 0.9212849717105785
company entity found: 广东省->广东省能源集团有限公司
, sim = 0.9212187301398342
company entity found: 国家->国家电力投资集团有限公司
, sim = 0.9082963948292435


In [173]:
linkCompany(output, company_ch_df, model, 0.9)

company entity found: 深圳->深圳金雅福控股集团有限公司
, sim = 0.9626824059668808
company entity found: 控股有限公司->中运富通控股集团有限公司
, sim = 0.93357818225778
company entity found: tcl->TCL集团股份有限公司
, sim = 0.9610930736775853
company entity found: 信息产业->中国电子信息产业集团有限公司
, sim = 0.9422591406677782
company entity found: 有限公司->中运富通控股集团有限公司
, sim = 0.9063590600834653
company entity found: 集团股份有限公司->中运富通控股集团有限公司
, sim = 0.9037290995606039
company entity found: 科技->山东创新金属科技有限公司
, sim = 0.9151182263571489
company entity found: 深圳市->深圳市投资控股有限公司
, sim = 0.931279784196003
company entity found: 控股->阿里巴巴集团控股有限公司
, sim = 0.9032541848685627
company entity found: 股份有限公司->中运富通控股集团有限公司
, sim = 0.906345197516979
company entity found: 陕西->陕西投资集团有限公司
, sim = 0.9387841332452491


## Link Degree

In [174]:
degree = {'degree': ['本科', '研究生', '博士']}
degree_df = pd.DataFrame(degree)
print(degree_df)

  degree
0     本科
1    研究生
2     博士


In [177]:
def preprocess_degree_entity_list(df, model):
    '''
    df: dafaframe
    model: word embedding model
    '''
    
    df['embeddings'] = ''
    for index, row in df.iterrows():
        name = row['degree']
        vec = model[name]
        df.set_value(index, 'embeddings', vec)

    print(df.head())
    return df

In [178]:
degree_df = preprocess_degree_entity_list(degree_df, model)

  degree                                         embeddings
0     本科  [-0.060282774, -0.077274054, 0.024572397, 0.00...
1    研究生  [-0.032758698, -0.03760286, 0.054133926, -0.05...
2     博士  [-0.06615932, 0.07013251, -0.006140913, -0.047...


c:\users\梁育诚\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # This is added back by InteractiveShellApp.init_path()


In [179]:
def linkDegree(output, degree_df, model, threshold):
    for term in output:
        if len(term) <= 1:
            continue
        if term in model.vocab:
            term_vec = model[term]
            company_candidate = dict()
            for index, row in degree_df.iterrows():
                name = row['degree']
                name_vec = row['embeddings']
                sim = calculate_cosine_similarity(term_vec, name_vec)
                if (sim > threshold):
                    company_candidate[name] = sim
            if len(company_candidate) == 0:
                continue
            company_candidate = sorted(company_candidate.items(), key=lambda item:item[1], reverse=True)
            print(f'degree entity found: {term}->{company_candidate[0][0]}, sim = {company_candidate[0][1]}')

In [181]:
linkDegree(output, degree_df, model, 0.95)

degree entity found: 博士->博士, sim = 1.0
